In [1]:
import os
#os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "0.6"  # preallocate a bit less memory so we can use pytorch next to jax

from time import time
import jax
import jax.numpy as jnp
import flax.linen as nn
import numpy as np
import optax
from typing import List, Dict
from jax.typing import ArrayLike
from meta_transformer import utils, preprocessing, torch_utils, module_path, on_cluster, output_dir
from meta_transformer.meta_model import MetaModel
from dataclasses import asdict
from meta_transformer.train import Updater, Logger
from meta_transformer.data import data_iterator, split_data
from etils import epath

/home/lauro/.virtualenvs/meta-models/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:

import pytest
import jax
import jax.numpy as jnp
import chex
from meta_transformer.preprocessing import augment_list_of_nets
import numpy.testing as npt
from meta_transformer import on_cluster, module_path, torch_utils
import os
import numpy as np


# load checkpoint data
if not on_cluster:
    dpath = os.path.join(module_path, "data/david_backdoors")  # local
else:
    dpath = "/rds/user/lsl38/rds-dsk-lab-eWkDxBhxBrQ/model-zoo/"  
dpath = os.path.join(dpath, "cifar10")
inputs_dir = os.path.join(dpath, "poison_noL1")
targets_dir = os.path.join(dpath, "clean")
architecture = torch_utils.CNNMedium()
inputs, targets, get_pytorch_model = torch_utils.load_pairs_of_models(
    model=architecture,
    data_dir1=inputs_dir,
    data_dir2=targets_dir,
    num_models=100,
    max_workers=None if on_cluster else 1,
)


rng = np.random.default_rng(42)
LAYERS_TO_PERMUTE = [f'Conv2d_{i}' for i in range(6)] + ['Dense_6']


def test_augment(nets):
    augmented = augment_list_of_nets(
        nets,
        rng,
        layers_to_permute=LAYERS_TO_PERMUTE,
    )
    assert len(augmented) == len(nets)
    assert len(augmented[0]) == len(nets[0])
    assert all(aug.keys() == net.keys() for aug, net in zip(augmented, nets))
    print()
    print()
    print()
    print()
    print()
    print(augmented[0].keys())

    print()
    print()
    print()
    print()
    print()


Loading pairs of models from:
/home/lauro/projects/meta-models/meta-transformer/data/david_backdoors/cifar10/poison_noL1
/home/lauro/projects/meta-models/meta-transformer/data/david_backdoors/cifar10/clean


In [4]:
test_augment(inputs)






dict_keys(['Conv2d_0', 'BatchNorm2d_1', 'Conv2d_2', 'BatchNorm2d_3', 'Conv2d_4', 'BatchNorm2d_5', 'Conv2d_6', 'BatchNorm2d_7', 'Conv2d_8', 'BatchNorm2d_9', 'Conv2d_10', 'BatchNorm2d_11', 'Linear_12', 'BatchNorm1d_13', 'Linear_14'])







/home/lauro/.virtualenvs/meta-models/lib/python3.10/site-packages/einops/parsing.py:136: RuntimeWarning: It is discouraged to use axes names that are keywords: in
  warnings.warn("It is discouraged to use axes names that are keywords: {}".format(name), RuntimeWarning)
